In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split 


from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV, learning_curve

In [2]:
data = pd.read_pickle('/Users/hansangjun/Desktop/Springboard/Capstone2/telco_data/fulldata.pkl')

In [3]:
data.drop(columns=['City', 'Zip Code'], inplace=True)

In [4]:
data.head()

,Gender,Senior Citizen,Partner,Dependents,Tenure Months,Phone Service,Multiple Lines,Internet Service,Online Security,Online Backup,...,Monthly Charge,Total Refunds,Total Extra Data Charges,Total Long Distance Charges,Total Revenue,Age,Under 30,Married,Number of Dependents,Population
0,Male,No,No,No,2,Yes,No,DSL,Yes,Yes,...,53.85,0.0,0,20.94,129.09,37,No,No,0,58198
1,Male,No,Yes,No,1,Yes,No,No,No internet service,No internet service,...,18.80,0.0,0,43.57,62.37,29,Yes,Yes,0,58198
2,Female,No,Yes,Yes,3,Yes,No,Fiber optic,No,Yes,...,80.00,0.0,0,57.54,298.84,49,No,Yes,3,58198
3,Female,No,Yes,No,59,Yes,Yes,Fiber optic,Yes,Yes,...,94.75,0.0,0,967.01,6564.66,60,No,Yes,0,58198
4,Male,No,No,No,5,Yes,No,Fiber optic,No,No,...,80.10,0.0,0,61.75,460.30,56,No,No,0,58198


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 46 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Gender                             7043 non-null   object 
 1   Senior Citizen                     7043 non-null   object 
 2   Partner                            7043 non-null   object 
 3   Dependents                         7043 non-null   object 
 4   Tenure Months                      7043 non-null   int64  
 5   Phone Service                      7043 non-null   object 
 6   Multiple Lines                     7043 non-null   object 
 7   Internet Service                   7043 non-null   object 
 8   Online Security                    7043 non-null   object 
 9   Online Backup                      7043 non-null   object 
 10  Device Protection                  7043 non-null   object 
 11  Tech Support                       7043 non-null   objec

In [15]:
# a quick data wrangling - Align value of category.
dtype_object = []

g = data.columns.to_series().groupby(data.dtypes).groups

for k, v in g.items():
    if k == object:
        for i in range(len(v)):
            dtype_object.append(v[i])

for i in range(len(dtype_object)):
    print("####" + dtype_object[i] + "####")
    print(data[dtype_object[i]].value_counts())
    print(" ")


####Gender####
Male      3555
Female    3488
Name: Gender, dtype: int64
 
####Senior Citizen####
No     5901
Yes    1142
Name: Senior Citizen, dtype: int64
 
####Partner####
No     3641
Yes    3402
Name: Partner, dtype: int64
 
####Dependents####
No     5416
Yes    1627
Name: Dependents, dtype: int64
 
####Phone Service####
Yes    6361
No      682
Name: Phone Service, dtype: int64
 
####Multiple Lines####
No                  3390
Yes                 2971
No phone service     682
Name: Multiple Lines, dtype: int64
 
####Internet Service####
Fiber optic    3096
DSL            2421
No             1526
Name: Internet Service, dtype: int64
 
####Online Security####
No                     3498
Yes                    2019
No internet service    1526
Name: Online Security, dtype: int64
 
####Online Backup####
No                     3088
Yes                    2429
No internet service    1526
Name: Online Backup, dtype: int64
 
####Device Protection####
No                     3095
Yes          

We have to delete column 'Quarter' because it only has one value: Q3.

In [6]:
data.drop(columns=['Quarter'], inplace=True)

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 45 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Gender                             7043 non-null   object 
 1   Senior Citizen                     7043 non-null   object 
 2   Partner                            7043 non-null   object 
 3   Dependents                         7043 non-null   object 
 4   Tenure Months                      7043 non-null   int64  
 5   Phone Service                      7043 non-null   object 
 6   Multiple Lines                     7043 non-null   object 
 7   Internet Service                   7043 non-null   object 
 8   Online Security                    7043 non-null   object 
 9   Online Backup                      7043 non-null   object 
 10  Device Protection                  7043 non-null   object 
 11  Tech Support                       7043 non-null   objec

## Dummy Variables
1. extract object type column names.
2. make sure deleting dependent variable.
3. create dummy variables

In [8]:
# churn label should be object type.
data['Churn Label'] = data['Churn Label'].astype(object)

In [9]:
# Extract object type column names from a new list.
names_dummies = data.select_dtypes(include=['object']).columns.to_list()

# Dependent variable doesn't have to be a dummy variable.
names_dummies.remove('Churn Label')
print(names_dummies)

['Gender', 'Senior Citizen', 'Partner', 'Dependents', 'Phone Service', 'Multiple Lines', 'Internet Service', 'Online Security', 'Online Backup', 'Device Protection', 'Tech Support', 'Streaming TV', 'Streaming Movies', 'Contract', 'Paperless Billing', 'Payment Method', 'Customer Status', 'Referred a Friend', 'Offer', 'Internet Type', 'Device Protection Plan', 'Premium Tech Support', 'Streaming Music', 'Unlimited Data', 'Under 30', 'Married']


In [10]:
# Create dummy variables.
df = pd.get_dummies(data, columns=names_dummies, prefix=names_dummies)

# Print the columns names
print(df.columns)

Index(['Tenure Months', 'Monthly Charges', 'Total Charges', 'Churn Label',
       'Churn Score', 'CLTV', 'Satisfaction Score', 'Number of Referrals',
       'Tenure in Months', 'Avg Monthly Long Distance Charges',
       'Avg Monthly GB Download', 'Monthly Charge', 'Total Refunds',
       'Total Extra Data Charges', 'Total Long Distance Charges',
       'Total Revenue', 'Age', 'Number of Dependents', 'Population',
       'Gender_Female', 'Gender_Male', 'Senior Citizen_No',
       'Senior Citizen_Yes', 'Partner_No', 'Partner_Yes', 'Dependents_No',
       'Dependents_Yes', 'Phone Service_No', 'Phone Service_Yes',
       'Multiple Lines_No', 'Multiple Lines_No phone service',
       'Multiple Lines_Yes', 'Internet Service_DSL',
       'Internet Service_Fiber optic', 'Internet Service_No',
       'Online Security_No', 'Online Security_No internet service',
       'Online Security_Yes', 'Online Backup_No',
       'Online Backup_No internet service', 'Online Backup_Yes',
       'Device Prote

In [11]:
df.head()

,Tenure Months,Monthly Charges,Total Charges,Churn Label,Churn Score,CLTV,Satisfaction Score,Number of Referrals,Tenure in Months,Avg Monthly Long Distance Charges,...,Premium Tech Support_No,Premium Tech Support_Yes,Streaming Music_No,Streaming Music_Yes,Unlimited Data_No,Unlimited Data_Yes,Under 30_No,Under 30_Yes,Married_No,Married_Yes
0,2,53.85,108.15,1,86,3239,1,0,2,10.47,...,1,0,1,0,0,1,1,0,1,0
1,1,18.80,18.80,0,51,5160,3,9,1,43.57,...,1,0,1,0,1,0,0,1,0,1
2,3,80.00,241.30,0,76,4264,3,2,3,19.18,...,0,1,1,0,0,1,1,0,0,1
3,59,94.75,5597.65,0,26,5238,3,4,59,16.39,...,1,0,1,0,0,1,1,0,0,1
4,5,80.10,398.55,0,22,5225,3,0,5,12.35,...,1,0,1,0,0,1,1,0,1,0


## Scale Data
1. collect int and float.
2. define scaler.
3. fit scale_df
4. transform the data using fitted scaler.
5. make it dataframe.

In [16]:
# collect int and float
names_list = df.select_dtypes(include=['float', 'int']).columns.to_list()

names_list_SS = []

for i in names_list:
    names_list_SS.append(i + "_SS")


scale_df = df[names_list]

# define scaler
scaler = StandardScaler()

# fit scale_df
scaler.fit(scale_df)

# transform the data using fitted scaler
scaled_df = scaler.transform(scale_df)

# make it dataframe
scaled_df = pd.DataFrame(scaled_df, columns=names_list_SS) 
scaled_df.head()

,Tenure Months_SS,Monthly Charges_SS,Total Charges_SS,Churn Score_SS,CLTV_SS,Satisfaction Score_SS,Number of Referrals_SS,Tenure in Months_SS,Avg Monthly Long Distance Charges_SS,Avg Monthly GB Download_SS,Monthly Charge_SS,Total Refunds_SS,Total Extra Data Charges_SS,Total Long Distance Charges_SS,Total Revenue_SS,Age_SS,Number of Dependents_SS,Population_SS
0,-1.236724,-0.362660,-0.958066,1.268402,-0.981675,-1.868323,-0.650409,-1.238238,-0.808503,0.023734,-0.362660,-0.248313,-0.2733,-0.860098,-1.014062,-0.567773,-0.486835,1.77114
1,-1.277445,-1.527580,-0.997486,-0.357720,0.642199,-0.203836,2.348606,-1.278988,1.334306,-1.004796,-1.527580,-0.248313,-0.2733,-0.833368,-1.037350,-1.045409,-0.486835,1.77114
2,-1.196004,0.506459,-0.899322,0.803796,-0.115215,-0.203836,0.016039,-1.197489,-0.244640,0.072712,0.506459,-0.248313,-0.2733,-0.816867,-0.954813,0.148681,2.629292,1.77114
3,1.084337,0.996690,1.463808,-1.519235,0.708135,-0.203836,0.682487,1.084470,-0.425257,-0.319109,0.996690,-0.248313,-0.2733,0.257395,1.232209,0.805430,-0.486835,1.77114
4,-1.114563,0.509783,-0.829946,-1.705078,0.697145,-0.203836,-0.650409,-1.115991,-0.686796,-0.368087,0.509783,-0.248313,-0.2733,-0.811894,-0.898457,0.566612,-0.486835,1.77114


In [17]:
# drop original int and float varaibles
df.drop(columns=names_list, inplace=True)

# concatenating scaled_df and df along columns
df = pd.concat([scaled_df, df], axis=1)

In [18]:
df.head()

,Tenure Months_SS,Monthly Charges_SS,Total Charges_SS,Churn Score_SS,CLTV_SS,Satisfaction Score_SS,Number of Referrals_SS,Tenure in Months_SS,Avg Monthly Long Distance Charges_SS,Avg Monthly GB Download_SS,...,Premium Tech Support_No,Premium Tech Support_Yes,Streaming Music_No,Streaming Music_Yes,Unlimited Data_No,Unlimited Data_Yes,Under 30_No,Under 30_Yes,Married_No,Married_Yes
0,-1.236724,-0.362660,-0.958066,1.268402,-0.981675,-1.868323,-0.650409,-1.238238,-0.808503,0.023734,...,1,0,1,0,0,1,1,0,1,0
1,-1.277445,-1.527580,-0.997486,-0.357720,0.642199,-0.203836,2.348606,-1.278988,1.334306,-1.004796,...,1,0,1,0,1,0,0,1,0,1
2,-1.196004,0.506459,-0.899322,0.803796,-0.115215,-0.203836,0.016039,-1.197489,-0.244640,0.072712,...,0,1,1,0,0,1,1,0,0,1
3,1.084337,0.996690,1.463808,-1.519235,0.708135,-0.203836,0.682487,1.084470,-0.425257,-0.319109,...,1,0,1,0,0,1,1,0,0,1
4,-1.114563,0.509783,-0.829946,-1.705078,0.697145,-0.203836,-0.650409,-1.115991,-0.686796,-0.368087,...,1,0,1,0,0,1,1,0,1,0


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 89 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   Tenure Months_SS                          7043 non-null   float64
 1   Monthly Charges_SS                        7043 non-null   float64
 2   Total Charges_SS                          7043 non-null   float64
 3   Churn Score_SS                            7043 non-null   float64
 4   CLTV_SS                                   7043 non-null   float64
 5   Satisfaction Score_SS                     7043 non-null   float64
 6   Number of Referrals_SS                    7043 non-null   float64
 7   Tenure in Months_SS                       7043 non-null   float64
 8   Avg Monthly Long Distance Charges_SS      7043 non-null   float64
 9   Avg Monthly GB Download_SS                7043 non-null   float64
 10  Monthly Charge_SS                   

In [20]:
df.to_pickle('df_preprocessed_full.pkl')